In [1]:
import pyproj
from  pyproj  import  CRS
from pyproj import Proj
import h5py
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm, colors
import glob
from mpl_toolkits.basemap import Basemap
from netCDF4 import Dataset
import pandas as pd
import gzip
from collections import defaultdict

In [2]:
# 设置空间分辨率
resolution = 10000

# 将WGS 84坐标（4326）转化为等距圆柱投影（4088）
crs = CRS.from_epsg(4326)
crs = CRS.from_string("epsg:4326")
crs = CRS.from_proj4("+proj=latlon")
crs = CRS.from_user_input(4326)
crs2 = CRS.from_epsg(4088)
crs2 = CRS.from_string("epsg:4088")
crs2 = CRS.from_proj4("+proj=latlon")
crs2 = CRS.from_user_input(4088)

transformer = pyproj.Transformer.from_crs(crs,crs2)
transformer_back = pyproj.Transformer.from_crs(crs2,crs)

In [3]:
# 获取数组的长和宽
def get_nlat_nlon_npaeqd(resolution):
    nlat, nlon =40000000/ resolution,40000000/ resolution
    nlat = np.int(nlat)
    nlon = np.int(nlon)
    return nlat, nlon
    
    
def get_nlat_nlon_cyl(resolution):
    nlat, nlon =40000000/ resolution,20000000/ resolution
    nlat = np.int(nlat)
    nlon = np.int(nlon)
    return nlat, nlon

# 获取用来存放数据的grid数组，及其对应的grid_num数组（用来存放一个格子中有多少重合点），以及对应的初始time数组
def get_swh_grid(nlat, nlon):
    swh_grid = np.full(shape=(nlon,nlat), fill_value=np.nan)
    swh_grid_num = np.zeros((nlon,nlat))
    swh_grid_time = np.full(shape=(nlon,nlat), fill_value=np.nan)
    return swh_grid, swh_grid_num, swh_grid_time

# 填充每个点的数值，将有重复数据的点标记到swh_grid_num中,最后求平均
def coincident_point_mean(value_array, grid_array, grid_num_array, projlats, projlons):
    for i in range(len(value_array)):
        x = int((projlons[i] )/resolution)
        y = int(projlats[i]/resolution)
        if grid_num_array[x][y] == 0:
            grid_array[x][y] = value_array[i]
            grid_num_array[x][y] += 1
        else:
            grid_array[x][y] += value_array[i]
            grid_num_array[x][y] += 1
    grid_array = grid_array / grid_num_array
    return grid_array

# def coincident_time_log(value_array, grid_array, grid_num_array, grid_time_array, projlats, projlons):
#     time_dict = {}
#     for i in range(len(value_array)):
#         x = int((projlons[i] )/resolution)
#         y = int(projlats[i]/resolution)
#         if grid_num_array[x][y] == 0:
#             hy_grid[x][y] = hy_value_array_masked[i]
#             hy_time_grid[x][y] = hy_time_array[i]
#             dict_name = str(x) + '+' + str(y)
#             hy_time_dict[dict_name] = {hy_time_grid[x][y]: hy_grid[x][y]}
#             grid_num_array[x][y] = 1
        
#         else:
#             dict_name = str(x) + '+' + str(y)
#             hy_time_dict[dict_name][hy_time_array[i]] = hy_value_array_masked[i]
#             grid_num_array[x][y] += 1
        
#     # 删除没有重合点的子字典
#     for key in list(hy_time_dict.keys()):
#         if len(hy_time_dict[key].items()) < 2 :
#             del hy_time_dict[key]

In [4]:
# hy_dir_path = r'G:\remote_sensing_data\HY-2B\2020\07\20200701_20200711'
hy_dir_path_01 = r'G:\remote_sensing_data\HY-2B\2020\07\20200701_20200711\20200711'
hyfiles = glob.glob(hy_dir_path_01 + '\*.nc')
hy_lon_array = np.array([])
hy_lat_array = np.array([])
hy_time_array = np.array([])
hy_value_array = np.array([])
hy_swhc_mask_array = np.array([])

for hyfile in hyfiles:
    with Dataset(hyfile, mode='r') as fh:
        lons = fh.variables['lon'][:]
        lats = fh.variables['lat'][:]
        swhc = fh.variables['swh_c'][:]
        time = fh.variables['time'][:]
        swhc_mask = swhc.mask
        hy_lon_array = np.append(hy_lon_array, lons)
        hy_lat_array = np.append(hy_lat_array, lats)
        hy_value_array = np.append(hy_value_array, swhc)
        hy_time_array = np.append(hy_time_array, time)
        hy_swhc_mask_array = np.append(hy_swhc_mask_array, swhc_mask)
    hy_value_array_masked = np.ma.array(hy_value_array, mask=hy_swhc_mask_array)
hy_df = pd.DataFrame([hy_lon_array, hy_lat_array, hy_time_array, hy_value_array_masked], index=['lon', 'lat', 'time', 'value'])

In [5]:
hy_projlats,hy_projlons = transformer.transform(hy_lat_array,hy_lon_array)
# 获取东西半球的nlat，nlon，is_grid_west,is_grid_num_west
hy_nlat, hy_nlon =get_nlat_nlon_cyl(resolution)
hy_grid, hy_num_grip, hy_time_grid = get_swh_grid(hy_nlat*2, hy_nlon*2)

In [6]:
hy_time_dict = {}
for i in range(len(hy_value_array_masked)):
    x = int((hy_projlons[i] )/resolution)
    y = int(hy_projlats[i]/resolution)
    if hy_num_grip[x][y] == 0:
        hy_grid[x][y] = hy_value_array_masked[i]
        hy_time_grid[x][y] = hy_time_array[i]
        dict_name = str(x) + '+' + str(y)
        hy_time_dict[dict_name] = {hy_time_grid[x][y]: hy_grid[x][y]}
        hy_num_grip[x][y] = 1
        
    else:
        dict_name = str(x) + '+' + str(y)
        hy_time_dict[dict_name][hy_time_array[i]] = hy_value_array_masked[i]
        hy_num_grip[x][y] += 1
        
'''# 删除没有重合点的子字典
for key in list(hy_time_dict.keys()):
    if len(hy_time_dict[key].items()) < 2 :
        del hy_time_dict[key]'''

<ipython-input-6-c951ff59db16>:6: UserWarning: Warning: converting a masked element to nan.
  hy_grid[x][y] = hy_value_array_masked[i]


'# 删除没有重合点的子字典\nfor key in list(hy_time_dict.keys()):\n    if len(hy_time_dict[key].items()) < 2 :\n        del hy_time_dict[key]'

In [7]:
hy_mean_grid = coincident_point_mean(hy_value_array_masked, hy_grid, hy_num_grip, hy_projlats,hy_projlons)

<ipython-input-3-54fb976da36e>:31: UserWarning: Warning: converting a masked element to nan.
  grid_array[x][y] += value_array[i]


# Cryosat 数据处理

In [8]:
# cy_dir_path = r'G:\remote_sensing_data\CRYOSAT-2\20200701_20200711'
cy_dir_path_01 = r"G:\remote_sensing_data\CRYOSAT-2\2020\07\20200711"
cyfiles = glob.glob(cy_dir_path_01 + '\*.nc')
cy_lon_array = np.array([])
cy_lat_array = np.array([])
cy_swh_array = np.array([])
cy_swh_mask_array = np.array([])
cy_time_array = np.array([])

for cyfile in cyfiles:
    with Dataset(cyfile, mode='r') as fh:
        lons = fh.variables['lon_01'][:]
        lats = fh.variables['lat_01'][:]
        swh = fh.variables['swh_ocean_01_ku'][:]
        time = fh.variables['time_cor_01'][:]
        swh_mask = swh.mask
        if np.array(swh_mask,dtype = bool).shape == ():
            swh_mask = np.full((len(swh), ), False)
        cy_lon_array = np.append(cy_lon_array, lons)
        cy_lat_array = np.append(cy_lat_array, lats)
        cy_swh_array = np.append(cy_swh_array, swh)
        cy_swh_mask_array = np.append(cy_swh_mask_array, swh_mask)
        cy_time_array = np.append(cy_time_array, time)
cy_swh_array_masked = np.ma.array(cy_swh_array, mask=cy_swh_mask_array)


In [9]:
'''cy_lower_index = []
for i in range(len(cy_swh_array_masked)):
    if cy_lat_array[i] > 60 or cy_lat_array[i] < -60:
        cy_lower_index.append(i)

cy_lat_array = np.delete(cy_lat_array, cy_lower_index)
cy_lon_array = np.delete(cy_lon_array, cy_lower_index)
cy_swh_array = np.delete(cy_swh_array, cy_lower_index)
cy_swh_mask_array = np.delete(cy_swh_mask_array, cy_lower_index)
cy_swh_array_masked = np.ma.array(cy_swh_array, mask=cy_swh_mask_array)'''

'cy_lower_index = []\nfor i in range(len(cy_swh_array_masked)):\n    if cy_lat_array[i] > 60 or cy_lat_array[i] < -60:\n        cy_lower_index.append(i)\n\ncy_lat_array = np.delete(cy_lat_array, cy_lower_index)\ncy_lon_array = np.delete(cy_lon_array, cy_lower_index)\ncy_swh_array = np.delete(cy_swh_array, cy_lower_index)\ncy_swh_mask_array = np.delete(cy_swh_mask_array, cy_lower_index)\ncy_swh_array_masked = np.ma.array(cy_swh_array, mask=cy_swh_mask_array)'

In [10]:
cy_projlats,cy_projlons = transformer.transform(cy_lat_array,cy_lon_array)

# 获取东西半球的nlat，nlon，cy_grid_west,cy_grid_num_west
cy_nlat, cy_nlon =get_nlat_nlon_cyl(resolution)
cy_grid, cy_num_grip, cy_time_grid = get_swh_grid(cy_nlat*2, cy_nlon*2)


In [11]:
cy_time_dict = {}
for i in range(len(cy_swh_array_masked)):
    x = int((cy_projlons[i] )/resolution)
    y = int(cy_projlats[i]/resolution)
    if cy_num_grip[x][y] == 0:
        cy_grid[x][y] = cy_swh_array_masked[i]
        cy_time_grid[x][y] = cy_time_array[i]
        dict_name = str(x) + '+' + str(y)
        cy_time_dict[dict_name] = {cy_time_grid[x][y]: cy_grid[x][y]}
        cy_num_grip[x][y] = 1
        
    else:
        dict_name = str(x) + '+' + str(y)
        cy_time_dict[dict_name][cy_time_array[i]] = cy_swh_array_masked[i]
        cy_num_grip[x][y] += 1
'''
# 删除没有重合点的子字典
for key in list(cy_time_dict.keys()):
    if len(cy_time_dict[key].items()) < 2 :
        del cy_time_dict[key]'''

<ipython-input-11-893a831ee7fa>:6: UserWarning: Warning: converting a masked element to nan.
  cy_grid[x][y] = cy_swh_array_masked[i]


'\n# 删除没有重合点的子字典\nfor key in list(cy_time_dict.keys()):\n    if len(cy_time_dict[key].items()) < 2 :\n        del cy_time_dict[key]'

In [12]:
cy_mean_grid = coincident_point_mean(cy_swh_array_masked, cy_grid, cy_num_grip, cy_projlats,cy_projlons)

<ipython-input-3-54fb976da36e>:31: UserWarning: Warning: converting a masked element to nan.
  grid_array[x][y] += value_array[i]


In [34]:
len(cy_time_dict.keys())

42165

# 对比交叉点


In [13]:
# 将两个grid数组相减，还有数据的部分为交叉点
hy_cy_grid = hy_mean_grid - cy_mean_grid
hy_cy_diff_grid = np.argwhere(np.isnan(hy_cy_grid)!= True)

In [33]:
# 如果x大于2000或者y大于4000，x-4000，y-8000
for ij in hy_cy_diff_grid:
    print(ij)

[  27 6588]
[  31 1010]
[  32 1010]
[  33 1010]
[  34 1010]
[  67 6308]
[  68 6308]
[  69 6308]
[  70 6308]
[  71 6308]
[ 106 6027]
[ 107 6027]
[ 108 6027]
[ 109 6027]
[ 110 6027]
[ 133 6954]
[ 134 6954]
[ 135 6954]
[ 136 6954]
[ 152 1852]
[ 153 1852]
[ 154 1852]
[ 155 1852]
[ 174 7234]
[ 175 7234]
[ 176 7234]
[ 178 7787]
[ 182 7788]
[ 183 7788]
[ 184 7788]
[ 185 7788]
[ 187 7789]
[ 189 7789]
[ 211 7515]
[ 212 7515]
[ 213 7515]
[ 214 7515]
[ 229 6411]
[ 230 6411]
[ 231 6411]
[ 232 6411]
[ 233 6411]
[ 251 7795]
[ 267 6692]
[ 268 6692]
[ 272 7521]
[ 273 7521]
[ 275 7522]
[ 276 7522]
[ 277 7522]
[ 278 7522]
[ 279 7522]
[ 354 7254]
[ 355 7254]
[ 357 7255]
[ 358 7255]
[ 359 7255]
[ 360 7255]
[ 491 1851]
[ 492 1851]
[ 517 1571]
[ 536 6449]
[ 537 6449]
[ 538 6449]
[ 575 7629]
[ 576 7629]
[ 578 6179]
[ 579 6179]
[ 580 6179]
[ 613 1910]
[ 615 1911]
[ 616 1911]
[ 645 1640]
[ 646 1640]
[ 647 1640]
[ 648 1641]
[ 664 7336]
[ 665 7336]
[ 687 7608]
[ 688 7607]
[ 689 7607]
[ 710 7587]
[ 711 7587]
[788

In [50]:
hy_time_dict['-600+45']

{647807938.48887: 2.7880000000000003}

In [49]:
cy_time_dict['-600+45']

{647784616.667111: 3.3040000000000003}

In [46]:
for i in range(len(hy_cy_diff_grid)): 
    diff_x = hy_cy_diff_grid[i][0]
    if diff_x > 2000: diff_x -= 4000
    diff_y = hy_cy_diff_grid[i][1]
    if diff_y > 4000: diff_y -= 8000
    key = str(diff_x) + '+' + str(diff_y)
    
    print(key)

27+-1412
31+1010
32+1010
33+1010
34+1010
67+-1692
68+-1692
69+-1692
70+-1692
71+-1692
106+-1973
107+-1973
108+-1973
109+-1973
110+-1973
133+-1046
134+-1046
135+-1046
136+-1046
152+1852
153+1852
154+1852
155+1852
174+-766
175+-766
176+-766
178+-213
182+-212
183+-212
184+-212
185+-212
187+-211
189+-211
211+-485
212+-485
213+-485
214+-485
229+-1589
230+-1589
231+-1589
232+-1589
233+-1589
251+-205
267+-1308
268+-1308
272+-479
273+-479
275+-478
276+-478
277+-478
278+-478
279+-478
354+-746
355+-746
357+-745
358+-745
359+-745
360+-745
491+1851
492+1851
517+1571
536+-1551
537+-1551
538+-1551
575+-371
576+-371
578+-1821
579+-1821
580+-1821
613+1910
615+1911
616+1911
645+1640
646+1640
647+1640
648+1641
664+-664
665+-664
687+-392
688+-393
689+-393
710+-413
711+-413
788+406
794+127
811+307
-677+1472
-676+1473
-675+1473
-674+-1875
-673+-1875
-658+-773
-656+371
-655+371
-652+1201
-651+1201
-650+1202
-649+1202
-648+-1604
-647+-1604
-642+-1052
-641+651
-641+-1052
-640+651
-626+-1332
-625+-1332
-623+93

In [26]:
hy_overlap_swh = np.array([])
cy_overlap_swh = np.array([])
diff_value = np.array([])
for i in range(len(hy_cy_diff_grid)):
    hy_overlap_swh = np.append(hy_overlap_swh, hy_grid[hy_cy_diff_grid[i][0], hy_cy_diff_grid[i][1]])
    cy_overlap_swh = np.append(cy_overlap_swh, cy_grid[hy_cy_diff_grid[i][0], hy_cy_diff_grid[i][1]])
    diff_value = np.append(diff_value, round(hy_cy_grid[hy_cy_diff_grid[i][0], hy_cy_diff_grid[i][1]], 8))


In [27]:
# 将每个交叉点的数值记入DataFrame

hy_cy_data={'HY-2B swh':hy_overlap_swh,
      'CRYOSAT swh':cy_overlap_swh,
      'Diff':diff_value
     }
hy_cy_df = pd.DataFrame(hy_cy_data)